In [31]:
import csv
from googleapiclient.discovery import build
import googleapiclient.errors
import pandas as pd
from tqdm import tqdm

In [32]:
from googleapiclient.discovery import build

#Function use to extract comments from youtube videos 

def video_comments(video_id, api_key='API KEY'):
    # empty list for storing comment date
    comments_data = []

    # creating youtube resource object
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    # retrieve youtube video results
    video_response = youtube.commentThreads().list(
        part='snippet,replies',
        videoId=video_id
    ).execute()

    while video_response:
        # extracting required info
        # from each result object
        for item in video_response['items']:
            # Extracting comments
            comment_text = item['snippet']['topLevelComment']['snippet']['textDisplay']
            
            # Extracting comment date
            comment_date = item['snippet']['topLevelComment']['snippet']['publishedAt']
            
            # counting number of reply of comment
            replycount = item['snippet']['totalReplyCount']
            
            # if reply is there
            if replycount > 0:
                # iterate through all reply
                for reply in item['replies']['comments']:
                    # Extract reply
                    reply_text = reply['snippet']['textDisplay']
                    
                    # Extract reply date
                    reply_date = reply['snippet']['publishedAt']
                    
                    # Store comment data in list
                    comments_data.append([comment_text, comment_date])
            else:
                # If no replies, store comment data with empty reply fields
                comments_data.append([comment_text, comment_date])
        
        # Again repeat
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(
                part='snippet,replies',
                videoId=video_id,
                pageToken=video_response['nextPageToken']
            ).execute()
        else:
            break

    return comments_data


In [52]:
df= pd.read_csv("data/rubius/rubius.csv")
df= df
df

,Year,Video title,Posting date,Video id
0,2021,aprendiendo a conducir (mal),5 jun 2021,cXorRzBrjMs
1,2021,TRABAJANDO EN UNA HAMBURGUESERIA MUY EXTRAÑA |...,18 dec 2021,_HxEEc1uYDs
2,2022,ESTE JUEGO DA DEMASIADO MAL ROLLO | MADiSON FINAL,21 nov 2023,HHO36WzoPCI
3,2022,Animales siendo interesantes por 9 minutos 40 ...,13 aug 2022,R6j205V76Lo
4,2023,POV: hemos vuelto a 2018,4 nov 2023,DtJ2dB5NxGw
5,2023,EXPLOTANDO MI PC POR AÑO NUEVO | Fireworks Mania,31 dec 2023,6dtOY4rjm6Y
6,2023,me enterraron VIVO con ESO,16 dec 2023,WBp9zfXhryw
7,2023,RIVERS ME HACE PERDER LA PACIENCIA,24 aug 2023,ZV4nHALH4DY
8,2023,LA CAJA LEGENDARIA 2023,3 may 2023,Kqdgme7ZD-4
9,2023,11 minutos de los mejores fails de 2023,30 sep 2023,D6T8J0De16Q


In [53]:

titles = df.columns.values.tolist() #gets the name of the values of the df
titles.append('Comment')
titles.append('Comment date') #appends the names of the new values from the data to extract 
list_df = []
list_df.append(titles)

for i in tqdm(range(len(df))):
  item = df.iloc[i]
  video_id = item['Video id']
  comments_list = video_comments(video_id)

  for j in range(len(comments_list)):
    list_entry = []
    for c in df.columns:
      list_entry.append(item[c])
    list_entry.append(comments_list[j][0])
    list_entry.append(comments_list[j][1])
    list_df.append(list_entry)
    

#Print function has been commented due to the large size of data that will be processed. In case you want to visualize the output, remove the #. 
#print(list_df) 

100%|██████████| 12/12 [04:18<00:00, 21.56s/it]


In [54]:
#list_df
df= pd.DataFrame(list_df[1:], columns= list_df[0])
df.insert(0, "Youtuber", "Rubius") #Adds column with constant value, name of the youtuber.
df.to_csv("D:/CCiL/Inter_ling/data/comments/output_rubius.csv")

In [55]:
df

,Youtuber,Year,Video title,Posting date,Video id,Comment,Comment date
0,Rubius,2021,aprendiendo a conducir (mal),5 jun 2021,cXorRzBrjMs,lo del recuerdo fue hace 10 años,2024-01-01T06:10:54Z
1,Rubius,2021,aprendiendo a conducir (mal),5 jun 2021,cXorRzBrjMs,definitivamente este no vio cars,2023-12-30T18:47:06Z
2,Rubius,2021,aprendiendo a conducir (mal),5 jun 2021,cXorRzBrjMs,"<a href=""https://www.youtube.com/watch?v=cXorR...",2023-12-29T21:27:42Z
3,Rubius,2021,aprendiendo a conducir (mal),5 jun 2021,cXorRzBrjMs,"Temita <a href=""https://www.youtube.com/watch?...",2023-12-27T20:41:03Z
4,Rubius,2021,aprendiendo a conducir (mal),5 jun 2021,cXorRzBrjMs,Rubios reto una carrera de Xbox con pedales y ...,2023-12-23T05:05:10Z
...,...,...,...,...,...,...,...
23381,Rubius,2023,PERDIENDO LA CABEZA EN UN VIDEOJUEGO DE NUEVO,5 jan 2023,A-dy09g96YQ,Pp,2023-01-05T21:44:49Z
23382,Rubius,2023,PERDIENDO LA CABEZA EN UN VIDEOJUEGO DE NUEVO,5 jan 2023,A-dy09g96YQ,Aaaaa,2023-01-05T21:44:48Z
23383,Rubius,2023,PERDIENDO LA CABEZA EN UN VIDEOJUEGO DE NUEVO,5 jan 2023,A-dy09g96YQ,Locooooooo,2023-01-05T21:44:46Z
23384,Rubius,2023,PERDIENDO LA CABEZA EN UN VIDEOJUEGO DE NUEVO,5 jan 2023,A-dy09g96YQ,Nomaaa,2023-01-05T21:44:37Z
